In [1]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True' 

In [1]:
from ultralytics import YOLO
model = YOLO("yolo11x-obb.pt")
results = model.train(
    # data="data_combine.yaml",
    data = "data_com.yaml",
    name = "x_s512_best_args_comcombine_20ep",
    seed=44,
    epochs=200,
    
    batch=8,
    imgsz=512,
    patience=15,
    device="0",
    optimizer = "AdamW",
    lr0= 0.00088,
    lrf= 0.01134,   
    momentum= 0.82858,
    weight_decay= 0.0006,
    warmup_epochs= 2.33952,
    warmup_momentum= 0.58988,
    # box= 5.06927,
    box = 9.1,
    cls= 0.56241,
    dfl= 1.36767,
    hsv_h= 0.01707,
    hsv_s= 0.68688,
    hsv_v= 0.28102,
    degrees= 0.0,
    translate= 0.11263,
    shear= 0.0,
    perspective= 0.0,
    flipud= 0.45936,
    fliplr= 0.46892,
    bgr= 0.1,
    mosaic= 0.828,
    mixup= 0.0,
    copy_paste= 0.0,
    scale = 0.2,
    # scale = 0.8,
    dropout = 0.3,
    )

New https://pypi.org/project/ultralytics/8.3.22 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.18  Python-3.12.7 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12282MiB)
engine\trainer: task=obb, mode=train, model=yolo11x-obb.pt, data=data_com.yaml, epochs=200, time=None, patience=15, batch=8, imgsz=512, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=x_s512_best_args_comcombine_20ep3, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=44, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.3, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=N

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: woow070840 (waooang). Use `wandb login --relogin` to force relogin


Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLO11n...
AMP: checks passed 


train: Scanning E:\Workspace\SatellieteIMGDetection\dataset_aug\labels\train... 1685 images, 521 backgrounds, 0 corrupt: 100%|██████████| 2071/2071 [00:04<00:00, 426.51it/s] 

train: WARNING  E:\Workspace\SatellieteIMGDetection\dataset_aug\images\train\augment471.png: 2 duplicate labels removed
train: WARNING  E:\Workspace\SatellieteIMGDetection\dataset_aug\images\train\augment79.png: 1 duplicate labels removed


...

                   all        192       5916      0.707      0.572       0.61      0.391
EarlyStopping: Training stopped early as no improvement observed in last 15 epochs. Best results observed at epoch 113, best model saved as best.pt.
To update EarlyStopping(patience=15) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



128 epochs completed in 1.738 hours.
Optimizer stripped from runs\obb\x_s512_best_args_comcombine_20ep3\weights\last.pt, 118.3MB
Optimizer stripped from runs\obb\x_s512_best_args_comcombine_20ep3\weights\best.pt, 118.3MB

Validating runs\obb\x_s512_best_args_comcombine_20ep3\weights\best.pt...
Ultralytics 8.3.18  Python-3.12.7 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12282MiB)
YOLO11x-obb summary (fused): 483 layers, 58,737,913 parameters, 0 gradients, 202.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 24/24 [00:03<00:00,  7.02it/s]


                   all        192       5916      0.742      0.568      0.627      0.402
               ship(s)         79       4334      0.828      0.526      0.631      0.415
               ship(L)        131       1582      0.656      0.609      0.623      0.389
Speed: 0.2ms preprocess, 6.0ms inference, 0.0ms loss, 2.7ms postprocess per image
Results saved to runs\obb\x_s512_best_args_comcombine_20ep3


lr/pg0,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr/pg1,▂███▇▇▇▇▆▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁
lr/pg2,▇████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▁
metrics/mAP50(B),▁▂▁▂▃▄▃▄▂▂▄▄▄▅▂▅▃▄▆▅▆▅▇▇▅▆▇▆▆▇▇▇█▅▆▆█▇▆█
metrics/mAP50-95(B),▁▂▂▂▁▃▃▄▃▄▃▄▃▄▅▅▃▅▅▅▅▆▆▅▇▆▆▆▇▆▇▇▇▆▇▇█▇▇█
metrics/precision(B),▁▄▆▆▇▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇███▇▇
metrics/recall(B),▁▃▃▂▃▄▅▄▅▆▄▆▆▆▆▅▆▆▆▆▆▆▇▇▆▇▇▇▇▇▇▇▇▇▇█▇▇▇▇
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,███▇▇▆▆▆▅▅▄▄▄▄▃▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁


In [ ]:
# 하이퍼 파라미터 튜닝
from ultralytics import YOLO
model = YOLO("yolo11x-obb.pt")
args = {
    "box": 0.8,
    "cls": 0.5,
    "lr0": 0.001,
    "momentum": 0.937,
    "weight_decay": 0.0005,
    "warmup_epochs": 3.0,
    "warmup_momentum": 0.8,
    "warmup_bias_lr": 0.1,
    "hsv_h": 0.015,
    "hsv_s": 0.7,
    "hsv_v": 0.4,
    "degrees": 0.0,
    "translate": 0.1,
    "scale": 0.86,
    "shear": 0.0,
    "perspective": 0.0,
    "flipud": 0.5,
    "fliplr": 0.5,
    "mosaic": 1.0,
    "mixup": 0.0,
    
}

model.tune(
    data = "data_com.yaml", 
    epochs = 10, 
    imgsz = 512, 
    batch=12, 
    device = "cuda:0", 
    # cfg = "runs/obb/tune4/best_hyperparameters.yaml",
    iterations=100, 
    optimizer="AdamW", 
    name ="tunning_512_label", 
    **args, 
    # label_smoothing = 0.3,
    )

Tuner: Initialized Tuner instance with 'tune_dir=runs\obb\tune25'
Tuner:  Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/100 with hyperparameters: {'lr0': 0.001, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'box': 1.0, 'cls': 0.5, 'dfl': 1.5, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.86, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.5, 'fliplr': 0.5, 'bgr': 0.0, 'mosaic': 1.0, 'mixup': 0.0, 'copy_paste': 0.0}
Saved runs\obb\tune25\tune_scatter_plots.png
Saved runs\obb\tune25\tune_fitness.png

Tuner: 1/100 iterations complete  (614.66s)
Tuner: Results saved to runs\obb\tune25
Tuner: Best fitness=0.29188 observed at iteration 1
Tuner: Best fitness metrics are {'metrics/precision(B)': 0.65905, 'metrics/recall(B)': 0.48339, 'metrics/mAP50(B)': 0.52089, 'metrics/mAP50-95(B)': 0.26644, 'val/box_loss': 0.25665, 'val/cls_loss': 1.0

In [ ]:
# 직접 YOLO 클래스를 임포트
from ultralytics import YOLO

base = "yolo11x-obb.pt"
resume ="runs/obb/x_s512_bgr05_scale063/weights/best.pt"
# YOLO 모델 초기화
model = YOLO(base) 
results = model.train(
    data="data_com.yaml",
    name = "x_s512_bgr05_scale083_",
    epochs=500,
    batch=6,
    imgsz=512,
    patience=12,
    device="0",
    optimizer = "AdamW",
    lr0= 0.00092,
    lrf= 0.0114,
    momentum= 0.92132,
    weight_decay= 0.00046,
    warmup_epochs= 3.38007,
    warmup_momentum= 0.5953,
    box= 1.0,
    cls= 0.42928,
    dfl= 1.17353,
    hsv_h= 0.0173,
    hsv_s= 0.2954,
    hsv_v= 0.38414,
    degrees= 0.0,
    translate= 0.10367,
    shear= 0.0,
    perspective= 0.0,
    flipud= 0.40917,
    fliplr= 0.46613,
    bgr= 0.5,
    mosaic= 0.91853,
    mixup= 0.0,
    copy_paste= 0.0,
    scale = 0.2,
    seed = 0,
    dropout = 0.35,
    
    )

New https://pypi.org/project/ultralytics/8.3.23 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.18  Python-3.12.7 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12282MiB)
engine\trainer: task=obb, mode=train, model=yolo11x-obb.pt, data=data_com.yaml, epochs=500, time=None, patience=12, batch=6, imgsz=512, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=x_s512_bgr05_scale0832, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.35, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=F

Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLO11n...
AMP: checks passed 


train: Scanning E:\Workspace\SatellieteIMGDetection\dataset_aug\labels\train.cache... 1685 images, 521 backgrounds, 0 corrupt: 100%|██████████| 2071/2071 [00:00<?, ?it/s]

train: WARNING  E:\Workspace\SatellieteIMGDetection\dataset_aug\images\train\augment471.png: 2 duplicate labels removed
train: WARNING  E:\Workspace\SatellieteIMGDetection\dataset_aug\images\train\augment79.png: 1 duplicate labels removed



val: Scanning E:\Workspace\SatellieteIMGDetection\dataset_aug\labels\val.cache... 162 images, 30 backgrounds, 0 corrupt: 100%|██████████| 192/192 [00:00<?, ?it/s]


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     22/500      6.32G     0.2442      1.078      0.888          9        512: 100%|██████████| 346/346 [00:51<00:00,  6.70it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:02<00:00, 12.05it/s]

                   all        192       5916      0.621      0.494      0.494      0.255



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     23/500      5.63G     0.2423      1.036     0.9165        138        512:  18%|█▊        | 63/346 [00:09<00:41,  6.75it/s]

In [6]:
model = YOLO("runs\obb\combine_1024\weights/best.pt")

# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map  # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps  # a list contains map50-95 of each category

<>:1: SyntaxWarning: invalid escape sequence '\o'
<>:1: SyntaxWarning: invalid escape sequence '\o'
C:\Users\Seungwoo\AppData\Local\Temp\ipykernel_35504\4051414092.py:1: SyntaxWarning: invalid escape sequence '\o'
  model = YOLO("runs\obb\combine_1024\weights/best.pt")


Ultralytics 8.3.9  Python-3.12.7 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12282MiB)
YOLO11x-obb summary (fused): 483 layers, 58,737,913 parameters, 0 gradients, 202.7 GFLOPs


val: Scanning E:\Workspace\SatellieteIMGDetection\datasets\labels\val.cache... 92 images, 0 backgrounds, 0 corrupt: 100%|██████████| 92/92 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:14<00:00,  2.34s/it]


                   all         92       4910      0.877      0.688      0.782      0.591
               ship(s)         79       4334      0.913      0.699      0.781       0.58
               ship(L)         61        576      0.841      0.677      0.782      0.602
Speed: 1.1ms preprocess, 106.3ms inference, 0.0ms loss, 8.8ms postprocess per image
Results saved to runs\obb\val


array([    0.58025,     0.60218])

In [ ]:
# # YOLO 모델의 Detect 모듈을 제외한 백본만 추출
# backbone = nn.Sequential(*list(yolo.model.model.children())[:-1])
# backbone

In [ ]:
# head = nn.Sequential(list(yolo.model.model.children())[-1])
# head

In [1]:
# import torch
# import torch.nn as nn
# from ultralytics import YOLO

# # YOLO 모델 불러오기 (YOLOv5 예시)
# class YOLOv5WithCustomHead(nn.Module):
#     def __init__(self, yolo):
#         super(YOLOv5WithCustomHead, self).__init__()
#         self.yolo = yolo.model  # YOLO 모델의 모든 레이어
        
#         # YOLO의 중간층에서 사용할 레이어 인덱스를 확인해야 함
#         self.custom_head = nn.Sequential(
#             nn.Conv2d(256, 128, kernel_size=3, stride=1, padding=1),
#             nn.ReLU(),
#             nn.AdaptiveAvgPool2d((1, 1)),  # Global Average Pooling
#             nn.Flatten(),
#             nn.Linear(128, 10)  # 예시로 10개 클래스를 분류하는 헤드
#         )

#     def forward(self, x):
#         outputs = []  # 여러 텐서를 저장할 리스트
        
#         # 중간 피처 맵을 추출하는 부분
#         for i, layer in enumerate(self.yolo.model):
#             x = layer(x)
#             if isinstance(x, list):  # x가 리스트인 경우
#                 x = torch.cat(x, dim=1)  # 리스트의 텐서를 결합
#             if i == 9:  # 예시로 9번째 레이어 출력 사용
#                 feature_map = x  # 중간 피처 맵 추출
#                 outputs.append(feature_map)  # 필요한 피처 맵 저장

#         # 기존 YOLO 아웃풋 (객체 탐지)
#         yolo_output = x

#         # 새로운 헤드에서 중간 피처 맵 사용
#         custom_output = self.custom_head(feature_map)
        
#         return yolo_output, custom_output

# # 기존 YOLOv5 모델 불러오기
# yolo = YOLO('yolo11m.pt')

# # 커스텀 모델 생성 (YOLOv5에 새로운 헤드 추가)
# model_with_custom_head = YOLOv5WithCustomHead(yolo)

# # 예시 입력 데이터
# input_data = torch.randn(1, 3, 640, 640)

# # 모델 실행
# yolo_output, custom_output = model_with_custom_head(input_data)

# # 결과 출력
# print("YOLO Output Shape:", yolo_output.shape)
# print("Custom Head Output Shape:", custom_output.shape)



TypeError: cat() received an invalid combination of arguments - got (Tensor, int), but expected one of:
 * (tuple of Tensors tensors, int dim, *, Tensor out)
 * (tuple of Tensors tensors, name dim, *, Tensor out)


In [9]:
yolo.model

DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C3k2(
      (cv1): Conv(
        (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(192, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(256, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
   